# **Detection and pictogram classification**
## Template matching method based on Hu’s moments

#### **Libraries**

In [30]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
import os
import glob
import json

#### **Functions**

In [32]:
def opencv_show(image, legend):
    """ Display an image using OpenCV """
    cv.imshow(legend, image)
    cv.waitKey(0)  # Wait indefinitely for a key press
    cv.destroyAllWindows()  # Close all OpenCV windows

def matplotlib_show(image, legend, axis_mode='off'):
    """ Display an image using Matplotlib """
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.title(legend)
    plt.axis(axis_mode)
    plt.show()

def show_2figures(img1, img2, title1='', title2=''):
    # title must be = 'Some title'

    # Create a figure and axes
    _ , axes = plt.subplots(1, 2, figsize=(10, 5))

    # Display the images
    axes[0].imshow(cv.cvtColor(img1, cv.COLOR_BGR2RGB))
    axes[0].set_title(title1)
    axes[0].axis('off')

    axes[1].imshow(cv.cvtColor(img2, cv.COLOR_BGR2RGB))
    axes[1].set_title(title2)
    axes[1].axis('off')

    # Adjust the spacing between subplots
    plt.subplots_adjust(wspace=0.1)

def show_3figures(img1, img2, img3, title1='', title2='', title3=''):
    _ , axes = plt.subplots(1, 3, figsize=(16, 12))

    axes[0].imshow(cv.cvtColor(img1, cv.COLOR_BGR2RGB))
    axes[0].set_title(title1)
    axes[0].axis('off')

    axes[1].imshow(cv.cvtColor(img2, cv.COLOR_BGR2RGB))
    axes[1].set_title(title2)
    axes[1].axis('off')

    axes[2].imshow(cv.cvtColor(img3, cv.COLOR_BGR2RGB))
    axes[2].set_title(title3)
    axes[2].axis('off')

    plt.subplots_adjust(wspace=0.1)

def show_4figures_frame(img1, img2, img3, img4, title1='', title2='', title3='', title4=''):

    _ , axes = plt.subplots(2,2, figsize=(5,5))

    axes[0, 0].imshow(cv.cvtColor(img1, cv.COLOR_BGR2RGB))
    axes[0, 0].set_title(title1)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(cv.cvtColor(img2, cv.COLOR_BGR2RGB))
    axes[0, 1].set_title(title2)
    axes[0, 1].axis('off')

    axes[1, 0].imshow(cv.cvtColor(img3, cv.COLOR_BGR2RGB))
    axes[1, 0].set_title(title3)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(cv.cvtColor(img4, cv.COLOR_BGR2RGB))
    axes[1, 1].set_title(title4)
    axes[1, 1].axis('off')

    plt.subplots_adjust(wspace=0, hspace=0)

def plot_image_and_histogram(image, title='Image and Histogram'):
    """
    Plot an image and its histogram side by side.

    Parameters:
    - image: Input image (numpy array)
    - title: Title for the entire plot (default: 'Image and Histogram')
    - channel: Color channel to plot histogram for (default: 0 - first channel)

    Returns:
    - None (displays the plot)
    """
    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Plot the image
    # Grayscale image
    ax1.imshow(image, cmap='gray')
    ax1.set_title('Grayscale Image')
    ax1.axis('off')

    # Calculate histogram
    #hist = cv.calcHist([image], [0], None, [256], [0, 256])
    hist = cv.calcHist([image], [0], None, [256], [0, 256]).flatten()

    # Plot histogram
    ax2.plot(hist, color='blue')
    ax2.set_title(f'Histogram')
    ax2.set_xlabel('Pixel Value')
    ax2.set_ylabel('Pixel Count')

    # Set overall figure title
    fig.suptitle(title, fontsize=16)

    # Adjust layout and display
    plt.tight_layout()
    plt.show()

def remove_blobs_borders2(binary_image): 
    """
    Remove blobs (e todo o seu conteúdo) cujas bounding boxes tocam na borda da imagem.
    """
    
    contours, _ = cv.findContours(binary_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    h, w = binary_image.shape

    mask = binary_image.copy()

    for contour in contours:
        
        x, y, bw, bh = cv.boundingRect(contour)

        if x <= 0 or y <= 0 or (x + bw) >= w or (y + bh) >= h:
        
            mask[y:y+bh, x:x+bw] = 0

    num_labels, labels, stats, _ = cv.connectedComponentsWithStats(mask, connectivity=8)
    area_minima = 10  
    filtered = np.zeros_like(mask)
    for i in range(1, num_labels):  
        area = stats[i, cv.CC_STAT_AREA]
        if area >= area_minima:
            filtered[labels == i] = 255
    return filtered

def create_folder_if_not_exists(folder_path):
    """ 
    Creates a folder at the specified path if does not already exists

    Args:
        folder_path (str): The path to the folder to be created
    """
    if not os.path.exists(folder_path):
        try: 
            os.makedirs(folder_path)
            print(f"Folder '{folder_path}' created sucessfully")
        except OSError as e:
            print(f"Error creating folder '{folder_path}': {e}")
    else:
        print(f"Folder '{folder_path}' already exists.")

def gama_correction(image, gamma):
    image = image/255
    image = cv.pow(image, gamma)
    return np.uint8(image*255)  

def binarize_and_contours(img):
    
    #img = gama_correction(img,gamma=1.1)#1.42
    _,binary_image = cv.threshold(img, 120, 255, cv.THRESH_BINARY)
    final_image = remove_blobs_borders2(binary_image)
    contours, _ = cv.findContours(final_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    return contours, final_image

def calculate_hu_moments(img):

    moments = cv.moments(img)
    hu_moments = cv.HuMoments(moments)
    hu_moments_log = [-np.sign(m[0]) * np.log(abs(m[0])) if m[0] !=0 else 0 for m in hu_moments]

    return hu_moments_log

def roi(imagem):

    if imagem is None:
        print(f"Error: Could not read the image")
        return 0

    contornos, _ = cv.findContours(imagem, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    mascara = np.zeros_like(imagem)

    area_img = imagem.shape[0] * imagem.shape[1]

    for c in contornos:
        area = cv.contourArea(c)
        if area < 0.99 * area_img:  
            cv.drawContours(mascara, [c], -1, 255, -1) 
    
    inv = cv.bitwise_not(mascara)
    f = cv.bitwise_or(imagem,inv)
    #show_2figures(imagem,f)

    return f

def align(image):
    if image is None:
        print("Error: Could not read the image")
        return None

    try:
        contours, _ = cv.findContours(image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        if not contours:
            print("Warning: contours not found in the image")
            return None

        for contour in contours:
            rot_rect = cv.minAreaRect(contour)
            w, h = map(int, rot_rect[1])
            if w == 0 or h == 0:
                continue

            src_pts = cv.boxPoints(rot_rect).astype("float32")
            dst_pts = np.array([[0, h-1], [0, 0], [w-1, 0], [w-1, h-1]], dtype="float32")
            M = cv.getPerspectiveTransform(src_pts, dst_pts)
            objeto_crop = cv.warpPerspective(image, M, (w, h))

            return objeto_crop 

        print("Warning: Invalid object found")
        return None

    except Exception as e:
        print(f"Error: Alignment failure: {e}")
        return None

def contar_objetos(final_image):

    if final_image is None:
        print(f"Error: Image is None")
        return 0

    c, _ = cv.findContours(final_image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    mask_image = np.zeros_like(final_image)

    for cont in c:
        
        temp_mask = np.zeros_like(final_image)
        cv.drawContours(temp_mask, [cont], 0, 255, -1)

        mask_image = cv.bitwise_or(mask_image, temp_mask)

    mask_invertida = cv.bitwise_not(mask_image)
    mask = cv.bitwise_or(final_image,mask_invertida)

    fill_image = np.ones_like(mask)
    fill_image = cv.bitwise_not(fill_image)

    mask2 = cv.bitwise_xor(mask, fill_image)
    
    mask3 = cv.bitwise_and(mask,mask2)
    
    inv = cv.bitwise_not(mask)
    
    final_image = inv

    contours, hirearchy = cv.findContours(final_image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    holes = []
    max_circularity = 0
    if hirearchy is not None:
        #print(hirearchy[0])
        hirearchy = hirearchy[0]
        for i,(_,_,_, parent) in enumerate(hirearchy):
            if parent >=0:
                holes.append(contours[i])

                contour = contours[i]
                
                area = cv.contourArea(contour)
                perimeter = cv.arcLength(contour, True)

                #Circularidade
                circularity = (4* np.pi * area) / (perimeter ** 2) if perimeter !=0 else 0
                max_circularity = max(max_circularity, circularity)
              
    qt = len(holes) 
    #show_2figures( final_image, image_new,'bin',f"Quantidade de objetos: {len(holes)}")

    return qt, max_circularity

def weighted_hu_distances(hu_moments1, hu_moments2):
   
    hu1 = np.array([abs(float(val)) for val in hu_moments1], dtype=float)
    hu2 = np.array([abs(float(val)) for val in hu_moments2], dtype=float)

    #weights = np.array([1.0, 1.2, 1.0, 1.3, 0.1, 0.2, 0.1])
    #weights = np.array([1.0, 1.2, 1.0, 2.0, 0.3, 0.2, 0.1])
    weights = np.array([1.0, 1.5, 2.0, 2.0, 0.3, 0.2, 0.1])
   
    diff = hu1 - hu2
    weighted_diff = diff * weights
    distance = np.sqrt(np.sum(weighted_diff**2))
    return distance

def identify_class(hu_moments, classes_dict, qt_detectado=None,circ_detectado=None, max_distance=5.0):
    min_distance = float('inf')
    best_class = None
    best_qt = None
    best_circ = None

    for class_name, data in classes_dict.items():
        know_hu = data["hu"]
        expected_qt = data.get("qt", None)
        expected_circ = data.get("circ", None)

        euc_distance = weighted_hu_distances(know_hu, hu_moments)


        if euc_distance < min_distance:
            min_distance = euc_distance
            best_class = class_name
            best_qt = expected_qt
            best_circ = expected_circ

    if min_distance > max_distance:
        return "unknown", 0, qt_detectado
    
    if qt_detectado is not None and qt_detectado != best_qt:
        return "unknown", 0, qt_detectado
    
    if circ_detectado is not None and circ_detectado < best_circ:
        return "unknown", 0, qt_detectado

    confidence = 1.0 - (min_distance / max_distance)
    confidence = max(0.0, min(1.0, confidence))

    return best_class, confidence, qt_detectado

classes_objects = {
    "jude1":     {"hu":np.array([6.043951007685909,13.60208088762311,23.35835507428187,27.15066719660215,-52.4097678600345,35.50831033624038,54.75288679771986]),  "qt":4, "circ":0.8},
    "rowing1": {"hu":np.array([6.162908830265362,16.858396440486203,21.376478796589964,21.90578506362929,43.54899972679417,30.772665176004224,-46.28842154795458]), "qt":2, "circ":0.8},
    "gym1":  {"hu":np.array([6.297818485121514,16.396318044654567,20.800614217440316,26.315673770856478,-49.905932908181946,-34.53623025292398,-51.26242960574342]), "qt":3, "circ":0.8},
    "voleyball1": {"hu":np.array([6.107914454004844,16.186473869846495,21.79749584252538,23.267043595403607,-48.8548723521156,-32.72724873800065,45.800423580876306]),  "qt":3, "circ":0.8},
    "sailing1":     {"hu":np.array([6.321278302277117,16.044310492663314,22.731275040188788,22.857820016474708,-46.718720745575894,-30.97442598028512,45.71544181973173]),  "qt":4, "circ":0.35},
    "shooting1":     {"hu":np.array([6.168787392694332,14.67844751258439,26.72161339592639,21.943895149742936,-46.429508723052656,-29.97487781725709,-46.94367852576798]),  "qt":3, "circ":0.8},
    "dancing1":    {"hu":np.array([6.208480554293045,21.100549981073286,23.2927655877338,21.7944664386127,45.53353910402938,32.54362733872211,44.38608798442341]),   "qt":2, "circ":0.8},
    "running1":  {"hu":np.array([6.289214727357762,14.93898392537829,22.641186302209427,22.89291257438103,45.9138397986739,30.406543393851816,-46.12041878991688]),  "qt":2, "circ":0.8},
    "cycling1": {"hu":np.array([6.477566196335043,16.43500668177666,23.714247088707708,24.046188844946062,-47.98639408728539,-32.31521422383956,49.01633842946368]),   "qt":4, "circ":0.8},
    "boxing1":     {"hu":np.array([6.250775794080434,17.11442313139012,23.233321653224806,23.016046161928337,46.35696630388705,-31.740373929374915,46.664076038090556]),  "qt":2, "circ":0.5},
    "basquetball1":  {"hu":np.array([6.204920375753049,15.318663363231602,22.46273542632091,24.393430877814662,-47.8306282728356,-32.05371683420153,49.82844910332526]),  "qt":3, "circ":0.8},
    "archery1":     {"hu":np.array([6.222340510347874,14.854781749477112,24.585153671631822,23.32042698356853,-47.28318132728138,-31.498110258263246,-49.236004987751556]),  "qt":3, "circ":0.8},
    "fencing1":  {"hu":np.array([6.118100572492891,19.227811440796664,23.719461720554666,20.452590143078368,42.779470815494555,-30.102462950436713,-43.01942554977381]),"qt":2, "circ":0.8},
    "football1":  {"hu":np.array([6.2172208989947615,15.744392215941293,22.90702665483268,23.373027296269715,46.51976039619237,32.06967269667413,48.67219703846435]), "qt":3, "circ":0.8},
}

def process_image(image_path):
    image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error: Could not read the image in {image_path}")
        return
    
    image = gama_correction(image,gamma=1.42)
    contours, final_image = binarize_and_contours(image)

    image_new = cv.cvtColor(final_image, cv.COLOR_GRAY2BGR)

    counts = {"archery1": 0, "running1":0, "basquetball1":0, "boxing1":0, "cycling1":0,"fencing1":0, "football1":0, "dancing1":0, 
              "gym1":0, "jude1":0, "rowing1":0, "shooting1":0, "sailing1":0, "voleyball1":0, "unknown":0 }
    colors = {"archery1": (57, 255, 20), 
              "running1": (255,0,0), 
              "basquetball1": (0, 0, 128), 
              "boxing1": (128, 128,255), 
              "cycling1": (255, 0, 255), 
              "fencing1": (255, 255, 0), 
              "football1": (188, 74, 60), 
              "dancing1": (0, 100, 0), 
              "gym1": (255, 0, 128), 
              "jude1": (128, 0, 255), 
              "rowing1": (143, 0, 255),
              "shooting1": (169, 208, 142),
              "sailing1": (0, 128, 128), 
              "voleyball1": (128, 255, 255),
              "unknown":  (0, 0, 255)
                                            } 

    for i, contour in enumerate(contours):
        x, y, w, h = cv.boundingRect(contour)

        cropped_image = final_image[y:y+h, x:x+w]
        #cropped_image = roi(cropped_image)
        qt, circ = contar_objetos(cropped_image)

        cropped_image = align(cropped_image)      
        hu_moments = calculate_hu_moments(cropped_image)

        class_identified, confidence, qt2 = identify_class(hu_moments, classes_objects, qt_detectado=qt, circ_detectado=circ)

        counts[class_identified] += 1

        cv.drawContours(image_new, [contour], -1, colors[class_identified], 2)

        label = f"{class_identified}"
        cv.putText(image_new, label, (x, y-10), cv.FONT_HERSHEY_COMPLEX_SMALL, 2, (255,255,255), 3)
        
    #show_2figures(final_image, image_new)
    return final_image, image_new, counts

def contagens(lista_imagens):
    resultados = []

    for caminho in lista_imagens:
        nome = os.path.basename(caminho)
        _, _, contagens = process_image(caminho)

        total = sum(contagens.values())

        resultados.append({
            #"Image": nome,
            "Total number of unknown objects": contagens['unknown'],
            "Total number of objects that don't touch the edges": total,
            "Number of complete objects per category": contagens
        })

    return resultados

---

In [ ]:
imagens = glob.glob("img_level2/*")
for imagem in imagens:  
    nome = os.path.basename(imagem)
    final_image, image_new,_ = process_image(imagem)
    show_2figures(final_image, image_new,'',nome)

imagens = glob.glob("img_level2/*")
resultado = contagens(imagens)
print(json.dumps(resultado, indent=4, ensure_ascii=False))